# Кто выбирает президентов

# Часть 1: Парсим VK, учимся работать с API

- Коротко что это и зачем
- VK developers, API
- Создаём свое приложение, получаем токен 
- Первый запрос, выкачиваем свою стенку
- Список групп для парсинга, что хотим скачать и как
- Скачиваем всех людей, состоящих в группах
- Получаем соцдем
- Получаем стенки и записи

# Подготовка

Совсем скоро в России пройдут выборы президента и кандидаты вовсю проводят свои избирательные кампании. Один из способов общения со своим электоратом - это социальные сети, в которых можно найти официальные группы потенциальных президентов и почитать их программы, мысли и планы, чтобы сделать правильный выбор. Мы решили посмотреть, какие люди подписаны на группы кандидатов в VK, чем они живут, о чем пишут и что у них общего друг с другом кроме безграничного интереса к политике и посиделки в соцсеточке. 

К сожалению или к счастью, нельзя просто взять и нажать на одну кнопку, чтобы получить все данные из вк

<img align="center" src="pictures/csv.png" height="600" width="600">

Поэтому нас спасёт API.

Многие сайты и сервисы хотят упростить жизнь программистам и предоставляют им в пользование уже готовые куски кода, к которым можно просто обратиться и получить нужную информацию о сервисе. Такие куски кода называются API (Application programming interface). Вконтакте не является исключением.

Сразу же стоит сказать, что API Вконтакте это рай для исследователя. Можно скачать огромное количество всего без серьёзных ограничений. Например, Facebook по своему API отдаст вам информацию только о тех людях, которые установили ваше приложение, API Одноклассников потребует написать им письмо с объяснением того, зачем это вам понадобились их данные, а API Вконтакте отдаст вам всё практически задаром. Единственное, что он попросит — это не обращаться к сайту очень часто, и это очень круто.

Тем не менее, недавно Вконтакте провёл серьёзную реформу своей музыкальной части и монетизировал её. Это привело к изъятию куска API, связанного с музыкой из открытого доступа. Более того, существует опасность, что социальная сеть продолжит резать бесплатный функционал своего API после разных скандалов, связанных с банками, нагло собирающими информацию о пользователях для своих скоринговых моделей. В связи с этим, нужно успеть нарисёрчить по данным из соц сеток вдоволь, пока нас существенно не ограничили в их выгрузке.

Ознакомиться со способами взаимодействия с Вконтакте можно по [ссылке](https://vk.com/dev/manuals), о существовании которой вы даже не догадывались... По ней можно найти лучшего друга человека — понятную документацию. Для того, чтобы получить доступ к выгрузке данных, нужно пройти ряд бюрократических процедур.

Первая такая процедура заключается в создании своего приложения. Для этого переходим по [ссылке](http://vk.com/editapp?act=create) и проходимся по необходимым процедурам:

<img align="center" src="pictures/app_creation_1.png" height="600" width="600">

После подтверждения своей личности по номеру телефона, попадаем на страницу свежесозданного приложения
<img align="center" src="pictures/app_creation_2.png" height="600" width="600">

Слева нам будем доступна вкладка с настройками, перейдя в неё мы увидим все необходимые нам для работы с приложением параметры (хоть у нас и день открытых данных, ID приложения и токены все-таки замазаны)
<img align="center" src="pictures/app_creation_3.png" height="600" width="600">


Для работы с частью методов API этого вполне достаточно (обычно в заголовке такого метода стоит соответствующая пометка). Для части методов, используемых нами, может понадобиться ключ доступа.

Для того, чтобы получить его, необходимо сделать ещё пару странных манипуляций:

Переходим по ссылке вида (на месте звездочек должен стоять ID созданного вами приложения):

> https://oauth.vk.com/authorize?client_id=**********&scope=14&redirect_uri=https://oauth.vk.com/blank.html&display=page&v=5.16&response_type=token

<img align="center" src="pictures/app_creation_4.png" height="600" width="600">

В итоге по этому запросу будет сформирована ссылка следующего вида:
> https://oauth.vk.com/blank.html#access_token=25b636116ef40e0718fe4d9f382544fc28&expires_in=86400&user_id=*******

Первый набор знаков - access token, т.е. маркер доступа. Вторая цифра (expires_in=) время работы маркера доступа в секундах (одни сутки). По истечению суток нужно будет получить новый маркер доступа. Последняя цифра (user_id=) ваш ID Вконтакте. Нам в дальнейшем понадобится маркер доступа. Для удобства сохраним его в отдельном файле или экспортируем в глобальную область видимости. В целях безопасности ваших данных не стоит нигде светить токенами и тем более выкладывать их в открытый доступ.

Обратите внимание на ссылку, по которой мы делали запрос на предоставление токена. Внутри неё находится странный параметр scope=14. Эта загадочная цифра ничто иное, как права доступа к социальной сети. Подробнее познакомиться с взаимно-однозначным соответствием между числами и правами можно в документации. Например, если мы хотим получить доступ к друзьям, фото и аудио, мы подставим в scope цифру 2+4+8=14.

# Первые шаги

In [195]:
# Загружаем необходимые библиотеки

import datetime                  # Пакет для работы с временными форматами
import pickle                    # Пакет для подгрузки данных специфического для питона формата
import requests                  # Пакет для скачки данных из этих ваших интернетов
import pandas as pd              # Пакет для работы с таблицами
import numpy as np               # Пакет для работы с векторами и матрицами
import matplotlib.pyplot as plt  # Пакет для строительства графиков
import time            # Пакет для работы со временем. Например, помогает ставить заглушки 
                       # time.sleep(секунды), необходимые для того что ВК не банил нащего
                       # сборщика данных из-за слишком частых запросов
        
# Пакет для красивых циклов. При желании его можно отключить. Тогда из всех циклов придётся 
# удалять команду tqdm_notebook.
from tqdm import tqdm_notebook   # подробнее: https://github.com/tqdm/tqdm

In [148]:
# мой номер странички
my_user_id = 91857120
# версия используемого API
version = '5.73' 
# подгружаем токен
with open('token.txt') as f:
    token = f.read()

Чтобы задать вопрос API вконтакте, нужно сформировать ссылку следующего вида:

> https://api.vk.com/method/METHOD_NAME?PARAMETERS&access_token=ACCESS_TOKEN&v=API_VERSION

> https://название_api/используемый_метод?параметры_метода&другие_опции&ключ_доступа&версия_API

В ссылку сначала встраивается метод, с помощью которого мы осуществляем запрос и какие-то параметры для этого метода. Список всех существующих методов и параметров можно изучить, конечно же, в документации.

Попробуем узнать своё имя. Метод `users.get` возвращает расширенную информацию о пользователе. Параметр `user_ids` отвечает за `id` пользователей, информацию о которых нам хотелось бы вытащить.

In [14]:
url = "https://api.vk.com/method/users.get?user_ids={user_id}&access_token={token}&v={version}"
url = url.format(user_id=my_user_id, token=token, version=version)

In [46]:
response = requests.get(url) 
response

<Response [200]>

Ура! Благословенный 200-ответ! Мы получили его в удобном json формате, который python воспринимает как словарик.

In [18]:
name = response.json()
name

{'response': [{'first_name': 'Дмитрий',
   'id': 91857120,
   'last_name': 'Сергеев'}]}

Со словарями очень удобно работать. Круто, когда можно узнать как тебя зовут с комфортом.

In [19]:
name['response'][0]

{'first_name': 'Дмитрий', 'id': 91857120, 'last_name': 'Сергеев'}

Оформим всё, что мы сделали выше в виде удобной функции, которая будет пригодна для любых методов и параметров.

In [20]:
def vkDownload(method, parameters, token=token, version=version):
    """
        Возвращает результат запроса по методу
        
        method: string
            метод из документации, который хотим использовать
            
        parameters: string
            параметры используемого метода
            
        token: string
            токен Oauth доступа
        
        version: string
            версия API
    """
    
    # составляем ссылку
    url = 'https://api.vk.com/method/{method}?{parameters}&access_token={token}&v={version}'
    url = url.format(method=method, parameters=parameters, token=token, version=version)
    # запрашиваем ссылку и переводим в json (словарь)
    response = requests.get(url).json()
    
    return response

Снова скачаем имя пользователя, чтобы убедиться, что всё работает

In [47]:
vkDownload('users.get','user_ids=91857120')

{'response': [{'first_name': 'Дмитрий',
   'id': 91857120,
   'last_name': 'Сергеев'}]}

Попробуем выкачать записи со стены

In [24]:
wall = vkDownload('wall.get','user_ids=91857120')

In [29]:
# обратимся к первой записи, которая лежит в поле response по ключу items
wall['response']['items'][0]

{'attachments': [{'link': {'caption': 'habrahabr.ru',
    'description': 'Новогодние праздники — прекрасный повод попрокрастинировать в уютной домашней обстановке и вспомнить дорогие сердцу мемы из 2k17, уходящие навсегда, как совесть...',
    'photo': {'album_id': -2,
     'date': 1516015158,
     'height': 150,
     'id': 456239920,
     'owner_id': 91857120,
     'photo_130': 'https://pp.userapi.com/c840433/v840433818/44e72/KHvVSy49L4I.jpg',
     'photo_604': 'https://pp.userapi.com/c840433/v840433818/44e73/lE-VT-vioog.jpg',
     'photo_75': 'https://pp.userapi.com/c840433/v840433818/44e71/3mnh_DxG2lM.jpg',
     'text': '',
     'width': 150},
    'title': 'Парсим мемы в питоне: как обойти серверную блокировку',
    'url': 'https://habrahabr.ru/company/ods/blog/346632/'},
   'type': 'link'}],
 'can_delete': 1,
 'can_pin': 1,
 'comments': {'can_post': 1, 'count': 1, 'groups_can_post': True},
 'date': 1516015158,
 'from_id': 91857120,
 'id': 1957,
 'likes': {'can_like': 0, 'can_publis

В принципе это всё. Дальнейшая стратегия будет такой: открываем документацию, находим нужные метод и параметры и применяем их.

# Забираем всех пользователей их групп кандидатов

In [30]:
 candidate_groups ={
     'Навальный' : 'teamnavalny',
     'Жириновский' : 'liberal_democratic_party',
     'Грудинин' : 'grudininlive',
     'Путин' : 'vladimir_vladimirovichp',
     'Собчак' : 'kandidatprotivvseh',
     'Бабурин' : 'baburin2018',
     'Явлинский' : 'yabloko_ru',
     'Титов' : 'partrost',
     'Сурайкин' : 'komross'
    }

In [52]:
def getGroupMembers(group_id):
    """
        Возвращает список всех пользователей данной группы
        Итерации идут батчами по 1000 пользователей
        
        group_id: string
            идентификатор группы (ссылка)
    """
    
    # Узнаём число запросов, которое надо сделать 
    count = vkDownload('groups.getMembers','group_id=' + group_id)['response']['count']
    
    # выясняем, сколько запросов нам понадобиться
    n = int(np.ceil(count/1000))  
    
    # вектор, где мы будем хранить id пользователей
    members = []     
    
    for i in tqdm_notebook(range(n)): 
        # при помощи метода groups.getMembers получаем пользователей группы
        current_members = vkDownload('groups.getMembers','group_id='+group_id+'&offset='+str(1000*i))
        members.extend(current_members['response']['items'])
        # перед следующим запросом немножко подождем
        time.sleep(0.4)
        
    return members

Попробуем выкачать кандидата с минимальным числом подписчиков, - Бабурина (для скорости)

In [53]:
baburin_members = getGroupMembers(candidate_groups['Бабурин'])

In [55]:
len(baburin_members)

2837

In [56]:
baburin_members[:10]

[16474,
 31069,
 521638,
 570735,
 731932,
 738809,
 804029,
 1030041,
 1128654,
 1209417]

Прекрасно, получили список из 2837 идентификаторов пользователей, подписанных на группу Бабурина. Всех остальных кандидатов можно выгрузить аналогично (код приведен ниже), но сейчас воспользуемся уже готовым набором данных и просто подгрузим его.

In [ ]:
# # Цикл для выгрузки всех участников групп по каждому кандидату
# # Выкачиваем членов каждой группы 
# candidate_people = {}
# for candidate, group_id in candidate_groups.items():
#     candidate_people[candidate] = getGroupMembers(group_id)

# Сохраняем словарик с id пользователей
# with open('candidate_people', 'wb') as f:
#     pickle.dump(candidate_people, f)

In [82]:
# Подгружаем словарь с пользователями
with open('candidate_people', 'rb') as f:
    candidate_people = pickle.load(f)

Посмотрим на число подписчиков у каждого кандидата (и Навального)

In [83]:
for candidate, people in candidate_people.items():
    print("Кандидат: {}, число подписчиков: {}".format(candidate, len(people)))

Кандидат: Навальный, число подписчиков: 184099
Кандидат: Жириновский, число подписчиков: 102186
Кандидат: Грудинин, число подписчиков: 23913
Кандидат: Путин, число подписчиков: 679822
Кандидат: Собчак, число подписчиков: 40300
Кандидат: Бабурин, число подписчиков: 2768
Кандидат: Явлинский, число подписчиков: 14490
Кандидат: Титов, число подписчиков: 17404
Кандидат: Сурайкин, число подписчиков: 3822


А также составим список из уникальных пользователей и посмотрим, сколько их у нас всего

In [84]:
# объединяем всех подписчиков в один лист
unique_people = sum(candidate_people.values(), [])
# берем только уникальных людей
unique_people = list(set(unique_people))

len(unique_people)

1048855

Отлично, 1 048 855 уникальных пользователей, подписанных на кандидатов в президенты, у нас есть. Теперь скачаем немножко соцдема по этим людям.

# Соц-дем пользователей

In [94]:
# Основные параметры, которые мы хотим получить
fields = 'bdate, city, home_town, sex'

Для выкачки соц-дема будем использовать метод `users.get`. За раз можно скачать информацию о 100 пользователях. Если на запрос будет уходить секунда, то скачка соц-дема займет около 3 часов.

In [87]:
round(1048855/100/60/60, 2)

2.91

Для пробы пера возьмем первых (по дате регистрации в вк) 10 человек, подписанных на многоуважаемого Владимира Вольфовича

In [96]:
LDPR = candidate_people['Жириновский'][:10]
# объединяем лист с int в строку
LDPR = ','.join([str(user) for user in LDPR])

In [97]:
vkDownload('users.get', 'user_ids={}&fields={}'.format(LDPR, fields))

{'response': [{'first_name': 'Анатолий',
   'home_town': '',
   'id': 510,
   'last_name': 'Карпенко',
   'sex': 2},
  {'deactivated': 'deleted',
   'first_name': 'DELETED',
   'id': 1857,
   'last_name': '',
   'sex': 2},
  {'city': {'id': 2, 'title': 'Санкт-Петербург'},
   'first_name': 'Сергей',
   'home_town': 'Ленинград',
   'id': 2703,
   'last_name': 'Иванов',
   'sex': 2},
  {'bdate': '18.2.1988',
   'city': {'id': 2, 'title': 'Санкт-Петербург'},
   'first_name': 'Антон',
   'id': 3386,
   'last_name': 'Тузяк',
   'sex': 2},
  {'bdate': '4.3.1987',
   'city': {'id': 2, 'title': 'Санкт-Петербург'},
   'first_name': 'Sergey',
   'id': 6302,
   'last_name': 'Chulkov',
   'sex': 2},
  {'bdate': '5.3.1989',
   'city': {'id': 1, 'title': 'Москва'},
   'first_name': 'Елена',
   'id': 6917,
   'last_name': 'Кейних',
   'sex': 1},
  {'bdate': '14.11.1986',
   'city': {'id': 2, 'title': 'Санкт-Петербург'},
   'first_name': 'Антон',
   'id': 8188,
   'last_name': 'Минаков',
   'sex': 2},


А тепреь обернём это в функцию, чтобы пройтись по всем нашим пользователям и выгрузить их соц-дем информацию

In [98]:
def getUserInformation():
    """
        Возвращает словарь с выгруженными социально-демографическими данными 
        по всем пользователям, подписанным на кандидатов
    """
    
    # инициализируем пустой словарь, где будут хранитсья пользователи с их соцдемом
    candidate_people_information = {}
    # проходимся по всем кандидатам
    for candidate, people in candidate_people.items():
        # текущий список пользователей с соц-демом
        current_info = []
        # Выяснили число запросов
        n = int(np.ceil(count/1000))    
        
        for i in tqdm_notebook(range(n)):
            # берем текущий срез подписчиков
            ids = people[i*100:(i+1)*100]
            ids = ','.join([str(user) for user in ids])
            # выгружаем их и берем данные
            info = vkDownload('users.get','user_ids={}&fields={}'.format(ids, fields))
            info = info['response']
            # записываем в текущий лист
            current_info.extend(info)
            # ждем перед следующим запросом
            time.sleep(0.4)
        
        # записываем в финальный словарь
        candidate_people_information[candidate] = current_info
    
    return candidate_people_information

 Код готов, выполнять его я, конечно, не буду

In [101]:
# candidate_people_information = getUserInformation()

In [102]:
# with open('candidate_people_information', 'wb') as f:
#     pickle.dump(candidate_people_information, f)

with open('candidate_people_information', 'rb') as f:
    candidate_people_information = pickle.load(f)

In [103]:
# А теперь преобразуем полученный словарь в удобный датафрейм для последующей работы

# инициализируем датафрейм
people_information_df = pd.DataFrame()

# идем по всем кандидатам и соц-дему их подписчиков
for candidate, people in candidate_people_information.items():
    # преобразуем в словарь в датафрейм
    df = pd.DataFrame(people)
    # добавляем столбец с городом
    df['city'] = df.city.apply(lambda x: x['title'] if x is not np.nan else np.nan)
    # добавляем столбец с городом
    df['candidate'] = candidate
    # добавляем преобразованный датафрейм к финальному
    people_information_df = people_information_df.append(df, ignore_index=True)

In [104]:
people_information_df.head()

,bdate,city,deactivated,first_name,home_town,id,last_name,sex,candidate
0,NaN,Санкт-Петербург,NaN,Николай,NaN,6,Дуров,2,Навальный
1,10.8,Łódź,NaN,Елисей,Санкт-Петербург,154,Замахов,2,Навальный
2,23.11.1981,Санкт-Петербург,NaN,Михаил,NaN,175,Захаренков,2,Навальный
3,NaN,NaN,banned,Галина,NaN,348,Румянцева,1,Навальный
4,3.5.1985,Санкт-Петербург,NaN,Сергей,Санкт-Петербург,374,Макеев,2,Навальный


In [105]:
people_information_df.tail()

,bdate,city,deactivated,first_name,home_town,id,last_name,sex,candidate
1068799,19.4.1970,NaN,NaN,Ол,NaN,471573939,Гус,2,Сурайкин
1068800,23.11.2001,NaN,NaN,Анонимус,,472291073,Анонимный,2,Сурайкин
1068801,7.11.1953,Пермь,NaN,Viva-La,NaN,473095996,Resistance,2,Сурайкин
1068802,3.3.2003,Новосибирск,NaN,Роман,новосибирск,473736153,Ковалёв,2,Сурайкин
1068803,8.8.1978,Москва,NaN,Максим,,474265309,Сурайкин,2,Сурайкин


In [109]:
people_information_df.shape

(1068804, 9)

In [110]:
people_information_df.to_csv('russia_20!8.csv', sep='\t')

# Последние 100 постов со стен пользователей

Соцдем собран, теперь переходим к самому интересному - о чем же пишут люди, добровольно подписавшиеся на будущих президентов этой страны. 

К сожалению, запросов у нас очень много, батчами по 1000 или хотя бы по 100 пользователей прогонять мы не можем, и если оставлять код как есть, то выкачивать записи мы будем примерно 5 суток подряд. Можно, конечно, и подождать, позаниматься другими делами, поработать, наконец. Но мы не ищем легких путей и не хотим работать, поэтому обратимся к замечательному методу `execute`, который позволяет обернуть 25 запросов к API внутри себя, что значительно ускорит работу скрипта до примерно 6 часов.

In [149]:
url = 'https://api.vk.com/method/execute?code=return[\
API.wall.get({"owner_id": "475285338","count":"100"}),\
API.wall.get({"owner_id": "475288625","count":"100"})];&access_token='+token+'&v='+version

In [150]:
res = requests.get(url).json()

In [151]:
list(res['response'][0])

['count', 'items']

Отлично, метод работает. Создадим пару функций, первая будет дробить наших пользователей на батчи по 25 человек, а вторая будет из них формировать ссылку для метода `execute`

In [159]:
def makeBatch(uids):
    """
        Возвращает лист листов из пользователей, батчами по 25 человек 
    """
    batches = [uids[i:i+25] for i in range(0,len(uids),25)]
    
    return batches 

def makeUrl(batch, token=token):
    """
        Создаёт ссылку для метода execute на основе батчей пользователей
    """
    # составляем ссылку из кусочков
    begin = 'https://api.vk.com/method/execute?code=return['
    end = '];&access_token='+token+'&v='+version
    middle = ''
    # для каждого пользователя из батча используем метод wall.get 
    for bt in batch:
        middle += 'API.wall.get({"owner_id": "'+str(bt)+'","count":"100"}),'
        
    return begin + middle[:-1] + end 

Используя эти функции соберем для каждого кандидата свой датафрейм с постами. При этом будем собирать только по 10000 пользователей с непустыми стенками, чтобы не так сильно раздувать наши датафреймы и избегать банов со стороны вконтакте, сохраняя при этом репрезентативность выборки

In [165]:
def getAllPosts(candidate):
    """
        Возвращает список словарей, в каждом их которых посты пользователей
    """
    
    # инициализируем список для хранения постов
    wall = []
    # берем всех людей текущего кандидата
    people = candidate_people[candidate]
    
    # рандомно перемешаем наших пользователей, чтобы обеспечить репрезентативность выборки
    np.random.shuffle(people)
    
    # создадим свой небольшой класс для кастомной ошибки 
    # на случай превышения лимита запросов
    class LimitExceeded(Exception):
        pass
    
    # батчи пользователей
    batches = makeBatch(people)
    
    # итерируем по батчам
    for item in tqdm_notebook(batches):
        # если насобирали 10 тысяч пользователей, останавливаемся
        if len(wall) < 10000:
            try:
                # пробуем выгрузить по батчу стенки пользователей
                res = requests.get(makeUrl(item))
                res = res.json()['response']
                
                # если нас забанили (все респонсы пусты), поднимем ошибку
                if res == [False]*25:
                    raise LimitExceeded("Превышено число запросов")
                
                # записываем полученные данные в список
                wall.extend(res)
                
            except LimitExceeded:
                # если забанили - подождем минутку, чтобы обновились лимиты запросов
                time.sleep(60)
                
            # в конце каждой итерации ждем 2 секунды, чтобы не словить бан
            time.sleep(2)
        else:
            break
            
    return wall

Еще один код код, который я выполнять не буду

In [180]:
# baburin = getAllPosts('Сурайкин')
# with open('candidate_wall_suraykin.pkl', 'wb') as f:
#    pickle.dump(cur_wl, f)

# подгрузим полученный датасет
with open('candidate_wall_suraykin.pkl', 'rb') as f:
    candidate_wall_suraykin = pickle.load(f)

Отлично, осталось перегнать полученный массив данных в датафрейм!

In [204]:
def wallToDF(candidate_wall):
    """
        Возвращает преобразованный датафрейм с распарсенными записями со стены
        
        candidate_wall: list
            лист с записями пользователей
    """
    
    # инициализируем словарь для записи значений
    wall_information = {}
    
    # елси текущая запись пустая, сразу возвращаем None
    if not candidate_wall:
        return None
    
    # Проходимся по информации из всех записей и собираем самое важное в фичи 
    for item in candidate_wall['items']:
        # проверяем, что текущий элемент не пустой
        if item:
            new_post = []
            # Поработаем с датой чуть более подробно. Преобразуем дату в удобочитаемый формат
            # пытаемся достать элемент из словаря по ключу, возвращаем False, если его нет
            real_date = item.get('date', False)
            # преобразуем в datetime, если не False, иначе np.NaN
            real_date = np.NaN if not real_date else datetime.datetime.fromtimestamp(real_date)
            # добавляем дату
            new_post.append(real_date)       

            # Нормальная дата для фичи "Когда сделан репостнутый пост"
            date = item.get('copy_post_date', False)
            date = np.NaN if not date else datetime.datetime.fromtimestamp(date)
            new_post.append(date)
            
            # тип записи post, copy, reply, postpone, suggest
            new_post.append(item.get('post_type', np.NaN))   
            # кто оставил запись 
            new_post.append(item.get('from_id', np.NaN))     
            
            # скидываем текст из перепостов и постов в один массив
            text = ""
            # на случай репоста, достаем его из внутреннего массива
            for inside_content in item:
                try:
                    text += inside_content['copy_history'][0]['text']
                    text += " "
                except:
                    continue
            # добавляем основной текст поста, если есть       
            text += item.get('text', "")
            new_post.append(text)                          # контент записи 
            new_post.append(item['likes']['count'])        # число лайков 
            new_post.append(item['reposts']['count'])      # число репостов 
            new_post.append(item['comments']['count'])     # число комментов 
            new_post.append(item['comments']['can_post'])  # можно ли оставлять комменты 


            # просмотры (если запись не старая), пропуск, если запись старая 
            views = item.get('views', False)
            views = np.NaN if not views else views.get('count', np.NaN) 
            new_post.append(views)
            
            # Истиный владелец записи (если это репост или запись оставлена)
            new_post.append(item.get('copy_owner_id', np.NaN))
            
            
            # Подробнее: https://vk.com/dev/objects/post_source
            # Через что создана запись 
            post_source = item.get('post_source', False)
            post_source_type = np.NaN if not post_source else post_source.get('type', np.NaN) 
            new_post.append(post_source_type)
    
            # С какой платформы создана запись, если это телефон
            post_source_platform = np.NaN if not post_source else post_source.get('platform', np.NaN) 
            new_post.append(post_source_platform)
            
            # Дополнительный контент у поста
            try:
                new_post.append([attach['type'] for attach in item['attachments']])
            except:
                new_post.append(None)   
                
            # Сохраняям в словарик, ключ - id записи 
            wall_information[item['id']] = new_post
        else:
            # если item пустой или False, то переходим на следующий элемент
            continue
        
    # превращаем всё, что насобирали в таблицу
    df = pd.DataFrame(wall_information).T
    df.columns = ['date', 'copy_post_date', 'post_type', 'from_id', 'text', 
          'likes', 'reposts', 'comments', 'can_comment', 'views', 'copy_owner_id', 'type', 
          'platform', 'attachments']
    # добавляем столбец с id юзера
    df['uid'] = item['owner_id']
    df['wall_cnt'] = candidate_wall['count']
    
    return df

In [210]:
# инициализируем пустой датафрейм
suraykin_dataframe = pd.DataFrame()

# проходимся по всем подписчикам кандидата
for item in tqdm_notebook(candidate_wall_suraykin):
    try:
        suraykin_dataframe = suraykin_dataframe.append(wallToDF(item), ignore_index=True)
    except:
        continue

In [213]:
# # сохраняем получившийся датафрейм 
# with open('suraykin_dataframe.pkl', 'wb') as f:
#     pickle.dump(df_yvl, f)

Ура, ребята, у нас всё работает! (но это неточно)

Осталось всего лишь собрать вот такие вот датафреймы еще 7 раз по всем кандидатам (и еще 1 разок для того-чьё-имя-нелья-называть). К сожалению, за такие наглые действия по сбору данных Вконтакте может и покарать баном, поэтому нам пришлось скачивать датасеты параллельно с разных аккаунтов, чтобы избежать ограничений по числу запросов.